In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [16]:
# Device-independent code
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

In [17]:
import numpy as np

# Function to read the file and extract pairs
def read_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.read().strip().split('\n')
    pairs = [[s for s in line.split(' -> ')] for line in lines]
    return pairs

def tokenize(sentence):
  return sentence.lower().split()

# Tokenize and build vocabularies
def build_vocab(pairs):
    eng_vocab = set()
    ita_vocab = set()
    for eng, ita in pairs:
        eng_vocab.update(tokenize(eng))
        ita_vocab.update(tokenize(ita))
    return eng_vocab, ita_vocab

In [18]:
# Path to your text file
file_path = 'eng_ita_v2.txt'

# Process the data
pairs = read_data(file_path)
english_vocab, italian_vocab = build_vocab(pairs)

# Creating word to integer mapping
eng_word2int = {word: i for i, word in enumerate(english_vocab)}
ita_word2int = {word: i for i, word in enumerate(italian_vocab)}

# Creating integer to word mapping
eng_int2word = {i: word for word, i in eng_word2int.items()}
ita_int2word = {i: word for word, i in ita_word2int.items()}

print('English vocabulary size:', len(english_vocab))
print('Italian vocabulary size:', len(italian_vocab))

English vocabulary size: 4997
Italian vocabulary size: 13673


In [19]:
# Example usage
eng_example = "Who are you"
ita_example = "chi sei tu"

# Encoding
eng_encoded = np.array([eng_word2int[word] for word in tokenize(eng_example)], dtype=np.int32)
ita_encoded = np.array([ita_word2int[word] for word in tokenize(ita_example)], dtype=np.int32)

print('English text encoded:', eng_encoded)
print('Italian text encoded:', ita_encoded)

# Decoding
print('Decoded English:', ' '.join([eng_int2word[i] for i in eng_encoded]))
print('Decoded Italian:', ' '.join([ita_int2word[i] for i in ita_encoded]))

English text encoded: [3080  111 3851]
Italian text encoded: [2025 4671   58]
Decoded English: who are you
Decoded Italian: chi sei tu


In [20]:
# Special tokens
PAD_TOKEN = "<PAD>"
EOS_TOKEN = "<EOS>"
SOS_TOKEN = "<SOS>"
UNK_TOKEN = "<UNK>"

# Update the function to create mappings to include the SOS token
def create_mappings(vocab):
    vocab = [PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, UNK_TOKEN] + sorted(vocab)
    word2int = {word: i for i, word in enumerate(vocab)}
    int2word = {i: word for word, i in word2int.items()}
    return word2int, int2word

# Update the vocabularies
eng_word2int, eng_int2word = create_mappings(english_vocab)
ita_word2int, ita_int2word = create_mappings(italian_vocab)

In [21]:
class TranslationDataset(Dataset):
    def __init__(self, pairs, eng_word2int, ita_word2int):
        self.pairs = pairs
        self.eng_word2int = eng_word2int
        self.ita_word2int = ita_word2int

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        eng, ita = self.pairs[idx]
        eng_tensor = torch.tensor([self.eng_word2int[word] for word in tokenize(eng)] + [self.eng_word2int[EOS_TOKEN]], dtype=torch.long)
        ita_tensor = torch.tensor([self.ita_word2int[word] for word in tokenize(ita)] + [self.ita_word2int[EOS_TOKEN]], dtype=torch.long)
        return eng_tensor, ita_tensor

# Custom collate function to handle padding
def collate_fn(batch):
    eng_batch, ita_batch = zip(*batch)
    eng_batch_padded = pad_sequence(eng_batch, batch_first=True, padding_value=eng_word2int[PAD_TOKEN])
    ita_batch_padded = pad_sequence(ita_batch, batch_first=True, padding_value=ita_word2int[PAD_TOKEN])
    return eng_batch_padded, ita_batch_padded

In [22]:
from torch.nn.utils.rnn import pad_sequence

# Create the dataset and DataLoader
translation_dataset = TranslationDataset(pairs, eng_word2int, ita_word2int)
batch_size = 64
translation_dataloader = DataLoader(translation_dataset, batch_size=batch_size, shuffle=True,  drop_last=True, collate_fn=collate_fn)

print("Translation samples: ", len(translation_dataset))
print("Translation batches: ", len(translation_dataloader))

Translation samples:  120746
Translation batches:  1886


In [23]:
# Example: iterating over the DataLoader
for eng, ita in translation_dataloader:
    print("English batch:", eng)
    print("Italian batch:", ita)
    break # remove this to iterate over the whole dataset

English batch: tensor([[4478, 4787,  827, 3924,    2,    0,    0,    0,    0,    0,    0],
        [2149, 1849, 4478, 2507, 2629,    2,    0,    0,    0,    0,    0],
        [2146, 4787,   43, 4467, 1853, 2998, 4855, 4392, 3134, 3436,    2],
        [2146, 4883, 1671, 2799, 2081,  632, 2866,    2,    0,    0,    0],
        [4426, 1016, 4690, 4467,  448, 4467, 1760,    2,    0,    0,    0],
        [4478, 4917, 2508,  402, 1444,    2,    0,    0,    0,    0,    0],
        [4407, 4842,  170, 4472,    2,    0,    0,    0,    0,    0,    0],
        [2303, 2308,  156, 3614, 4467, 1853,    2,    0,    0,    0,    0],
        [2146, 1325, 4416, 4478, 2378, 2055,    2,    0,    0,    0,    0],
        [2146,  305,   37, 3950, 2931, 3156,    2,    0,    0,    0,    0],
        [4478, 2772, 3413,    2,    0,    0,    0,    0,    0,    0,    0],
        [2146, 4416, 4478,  654, 4885,    2,    0,    0,    0,    0,    0],
        [2583, 4787, 1895,    2,    0,    0,    0,    0,    0,    0,    0

In [24]:
class SimpleAttention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size * 2, bias=False)
        self.v = nn.Linear(hidden_size * 2, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden: [layer*D num, batch size, hidden size]
        # encoder_outputs: [batch size, sequence length, hidden size]

        # we only have 1 layer and 1 direction in the decoder RNN, drop this dimension
        hidden = hidden.squeeze(0)  # [batch size, hidden size]

        # Repeat decoder hidden state across the sequence length
        hidden = hidden.unsqueeze(1).repeat(1, encoder_outputs.size(1), 1)

        # Concatenate the hidden state with encoder outputs
        combined = torch.cat((hidden, encoder_outputs), dim=2)

        # Compute the attention scores
        x = torch.tanh(self.attn(combined))
        attention = self.v(x).squeeze(2)
        attention_weights = F.softmax(attention, dim=1)

        # Compute the context as weighted sum of encoder outputs
        context_vector = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)

        return context_vector, attention_weights

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=num_layers,
                            batch_first=True, bidirectional=True)

    def forward(self, x):
        embedded = self.embedding(x)
        outputs, (hidden, cell) = self.lstm(embedded)

        # concatenate hidden states of the bi-directional RNN layer
        hidden = torch.cat((hidden[0,:,:], hidden[1,:,:]), dim=1).unsqueeze(0)
        cell = torch.cat((cell[0,:,:], cell[1,:,:]), dim=1).unsqueeze(0)

        return outputs, hidden, cell

class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.attention = SimpleAttention(hidden_size)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(hidden_size + embed_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.attention_matrix = []

    def forward(self, x, hidden, cell, encoder_outputs, store_attention=False):
        x = self.embedding(x)
        context_vector, attention_weights = self.attention(hidden, encoder_outputs)

        if store_attention:
          # Store attention weights for visualization
          self.attention_matrix.append(attention_weights.detach().cpu().numpy())

        lstm_input = torch.cat((x, context_vector.unsqueeze(1)), dim=2)

        out, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))
        out = self.fc(out).reshape(out.size(0), -1)
        return out, hidden, cell

    def reset_attention_matrix(self):
        self.attention_matrix = []

In [26]:
# Hyperparameters
eng_vocab_size = len(eng_word2int)
ita_vocab_size = len(ita_word2int)
embed_size = 256
hidden_size = 512
num_layers = 1

# Initialize the models
encoder = Encoder(eng_vocab_size, embed_size, hidden_size, num_layers).to(DEVICE)
decoder = Decoder(ita_vocab_size, embed_size, hidden_size*2, num_layers).to(DEVICE)

In [27]:
def translate(encoder, decoder, sentence, eng_word2int, ita_int2word, max_length=15):
    encoder.eval()
    decoder.eval()
    with torch.inference_mode():
        # Tokenize and encode the sentence
        input_tensor = torch.tensor([eng_word2int[word] for word in tokenize(sentence)]
                                    + [eng_word2int[EOS_TOKEN]], dtype=torch.long)
        input_tensor = input_tensor.view(1, -1).to(DEVICE)  # batch_first=True

        # Pass the input through the encoder
        encoder_outputs, encoder_hidden, encoder_cell = encoder(input_tensor)
        # Initialize the decoder input with the SOS token
        decoder_input = torch.tensor([[eng_word2int[SOS_TOKEN]]], dtype=torch.long)  # SOS
        # Initialize the hidden state of the decoder with the encoder's hidden state
        decoder_hidden, decoder_cell = encoder_hidden, encoder_cell

        decoder.reset_attention_matrix()
        # Decoding the sentence
        decoded_words = []
        last_word = torch.tensor([[eng_word2int[SOS_TOKEN]]]).to(DEVICE)
        for di in range(max_length):
            logits, decoder_hidden, decoder_cell = decoder(last_word, decoder_hidden,
                                  decoder_cell, encoder_outputs, store_attention=True)
            next_token = logits.argmax(dim=1) # greedy
            last_word = torch.tensor([[next_token]]).to(DEVICE)
            if next_token.item() == ita_word2int[EOS_TOKEN]:
                break
            else:
                decoded_words.append(ita_int2word.get(next_token.item()))

        return ' '.join(decoded_words)

In [28]:
# Example usage
# s/he likes music, listening to music,
# tom likes hot chocolate
# i want to go home now
# tom likes chocolate
# tom was right about that
# tom said he would not come
# this is a fun game

sentence = "tom likes hot chocolate"
translated_sentence = translate(encoder, decoder, sentence, eng_word2int, ita_int2word)
print("Translated:", translated_sentence)

Translated: sconfitto proteggere sbarazzartene considererà usciti ottuso flessibile annegarono autorità clicca siediti qualosa andarmene insufficiente rendo


In [29]:
import torch.optim as optim
import torch.nn as nn
import random

# Loss Function (exclude padding)
loss_fn = nn.CrossEntropyLoss(ignore_index=eng_word2int[PAD_TOKEN])

# Optimizers
encoder_optimizer = torch.optim.AdamW(encoder.parameters())
decoder_optimizer = torch.optim.AdamW(decoder.parameters())

# Number of epochs
num_epochs = 10

# Training Loop
encoder.train()
decoder.train()

for epoch in range(num_epochs):
    for i, (input_tensor, target_tensor) in enumerate(translation_dataloader):
        input_tensor, target_tensor = input_tensor.to(DEVICE), target_tensor.to(DEVICE)

        # Zero gradients of both optimizers
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        target_length = target_tensor.size(1)

        # Encoder
        encoder_output, encoder_hidden, encoder_cell = encoder(input_tensor)

        # Decoder
        decoder_input = torch.full((batch_size, 1), eng_word2int[SOS_TOKEN], dtype=torch.long).to(DEVICE)
        decoder_hidden = encoder_hidden
        decoder_cell = encoder_cell

        # Randomly select a word index from the target sequence
        random_word_index = random.randint(0, target_length - 1)

        loss = 0

        for di in range(target_length):
            logits, decoder_hidden, decoder_cell  = decoder(decoder_input, decoder_hidden, decoder_cell, encoder_output)
            #if di == random_word_index:
            #    loss = loss_fn(logits, target_tensor[:, di])
            #    break  # Only compute loss for the randomly selected word
            loss += loss_fn(logits, target_tensor[:,di])
            decoder_input = target_tensor[:, di].reshape(batch_size, 1)  # Teacher forcing

        # Backpropagation
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(encoder.parameters(), 2)
        #torch.nn.utils.clip_grad_norm_(decoder.parameters(), 2)

        encoder_optimizer.step()
        decoder_optimizer.step()

        if i % 100 == 0:  # Print loss every 10 batches
            print(f'Epoch {epoch}, Batch {i}, Loss: {loss.item() / target_length:.4f}')


Epoch 0, Batch 0, Loss: 9.5263


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_attention(attention_matrix, input_sentence, output_sentence):
    attention_matrix = attention_matrix.squeeze(1)

    fig, ax = plt.subplots(figsize=(10,10))
    attention_heatmap = sns.heatmap(attention_matrix, annot=True, ax=ax, cmap="viridis", fmt=".2f",
                                    xticklabels=input_sentence, yticklabels=output_sentence)
    plt.xlabel('Input Sequence')
    plt.ylabel('Output Sequence')
    plt.title('Attention Heatmap')
    plt.show()

attention_matrix = np.array(decoder.attention_matrix)
plot_attention(attention_matrix, tokenize(sentence), tokenize(translated_sentence))